## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Открываем файл с данными и изучаем общую информацию. 

Импортируем библиотеку pandas и откроем файл с данными как объект DataFrame.

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('datasets/data.csv')

Описание данных:

* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

Выведем на экран первые 10 строк таблицы.

In [3]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


По первым строкам таблицы можем заметить следующие проблемы:
* трудовой стаж и ежемесячный доход записаны с точностью до 6 знаков после запятой, что усложняет визуальное восприятие таблицы;
* трудовой стаж имеет отрицательные и аномально большие значения, что противоречит здравому смыслу;
* данные об образовании записаны в разном регистре, что может повлиять на поиск дубликатов;
* одинаковые цели получения кредита записаны разными фразами, например, "сыграть свадьбу" и "на проведение свадьбы".

Выведем на экран общую информацию о таблице.

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Здесь видим следующие проблемы:
* в данных о трудовом стаже и о ежемесячном доходе есть пропущенные значения;
* трудовой стаж и ежемесячный доход имеют вещественный тип, тогда как все остальные числовые данные - целочисленный.

### Вывод

Изучена общая информация о предоставленных данных. Обнаружены проблемы в данных, в виду чего необходимо провести предобработку прежде чем приступать к анализу.

### Шаг 2. Предобработка данных

### Обработка пропусков

#### Количественные переменные

Определим пропуски в количественных переменных. Таких переменых три: days_employed, dob_years, total_income. Остальные, даже если имеют числовой тип, являются по сути категориальными, так как принимают значение из конечного множества. Тем не менее, посмотрим на них все.

In [5]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Из описания видим пропущенные данные в количественных переменных:
* в столбцах days_employed и total_income есть пропущенные значения. 
* в столбце dob_years есть нулевые значения, хотя возраст клиента не может быть равным нулю. 

Здесь же видим новую проблему категориальных переменных:
* в столбце children присутствуют аномальные значения "-1" и "20".

Займемся пропусками в столбцах days_employed и total_income. Количество пропущенных значений в них совпадает. Проверим, связаны ли эти пропуски между собой.

In [6]:
data[data['total_income'].isnull()].count()

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64

Действительно, общий трудовой стаж и ежемесячный доход пропущены в одних и тех же строках. Это случайные пропуски - они скорее всего зависят друг от друга, но не зависят от других значений в собственных столбцах. Такие пропуски могут влиять на результат анализа, поэтому выкидывать строки с ними нежелательно. Убедимся, что это так. Посмотрим на сведения о задолженностях клиентов, для которых недостает информации.

In [7]:
data[data['total_income'].isnull()].groupby('debt')['debt'].count()

debt
0    2004
1     170
Name: debt, dtype: int64

Задолженность различается, поэтому мы не станем выбрасывать эти строки из анализа - в них может содержаться важная информация. Необходимо выбрать, чем заполнить пропуски. Рассмотрим тип занятости в целевых строках. 

In [8]:
data[data['total_income'].isnull()].groupby('income_type')['income_type'].count()

income_type
госслужащий         147
компаньон           508
пенсионер           413
предприниматель       1
сотрудник          1105
Name: income_type, dtype: int64

Среди этих клиентов нет безработных, то есть доход и трудовой стаж у них скорее всего имеются. Вероятно, эти пропуски возникли в результате технического сбоя. В таком случае заменить пропуски нулями нельзя, следует выбрать характерные значения. Воспользуемся медианой, так как это позволит снизить влияние возможных выбросов в данных. Значения возьмем из вывода метода describe().

Заполним пропуски в столбце с доходами.

In [9]:
data['total_income'] = data['total_income'].fillna(1.450179e+05)

Для трудового стажа также ограничимся медианой. Столбец полон аномальных значений, но для поставленной задачи это не сыграет роли.

In [10]:
data['days_employed'] = data['days_employed'].fillna(-1203.369529)

Проверим, что пропусков не осталось.

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Действительно не осталось!

Изучим нулевые значения в столбце dob_years. Посчитаем их количество и посмотрим на строки, содержащие такие значения.

In [12]:
data.loc[data['dob_years'] == 0]['dob_years'].count()

101

In [13]:
data.loc[data['dob_years'] == 0].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,-2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,-1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,397856.565013,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,-1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль
1149,0,-934.654854,0,среднее,1,женат / замужем,0,F,компаньон,0,201852.430096,покупка недвижимости
1175,0,370879.508002,0,среднее,1,женат / замужем,0,F,пенсионер,0,313949.845188,получение дополнительного образования
1386,0,-5043.219890,0,высшее,0,женат / замужем,0,M,госслужащий,0,240523.618071,сделка с автомобилем
1890,0,-1203.369529,0,высшее,0,Не женат / не замужем,4,F,сотрудник,0,145017.900000,жилье
1898,0,370144.537021,0,среднее,1,вдовец / вдова,2,F,пенсионер,0,127400.268338,на покупку автомобиля


Возраст не используется для поставленной задачи на анализ. Заполним и его медианным значением. Здравому смыслу не противоречит - даже пенсионер в некоторых профессиях может быть 42-летним.

In [14]:
data['dob_years'] = data['dob_years'].replace(0, 42)

Проверим, что нулевых значений не осталось.

In [15]:
data.loc[data['dob_years'] == 0]['dob_years'].count()

0

Количественные пропуски заполнены.

#### Категориальные переменные

Категориальные переменные в наших данных: children, education, education_id, family_status, family_status_id, gender, income_type, debt, purpose.

Часть из них мы изучили благодаря методу describe(). Столбцы education_id, family_status_id, debt не имеют проблем. Столбец children имеет аномальные значения. Тем не менее, посмотрим на список уникальных значений каждого из них. Будем использовать метод value_counts(), так как он выводит значения в столбик и позволяет сразу оценить масштаб проблемы.

Столбец children.

In [16]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Маловероятно, чтобы кто-то имел -1 ребенка или сразу 20. Такие пропуски похожи на технический сбой - человеческие ошибки скорее всего были бы более разнообразны. 

Аномальные значения составляют 0,6% от общего количества. Посмотрим на строки, содержащие эти значения, чтобы убедится, что они абсолютно случайны, и их можно убрать.

Изучим данные о клиентах с -1 ребенком.

In [17]:
data.loc[data['children'] == -1].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
291,-1,-4417.703588,46,среднее,1,гражданский брак,1,F,сотрудник,0,102816.346412,профильное образование
705,-1,-902.084528,50,среднее,1,женат / замужем,0,F,госслужащий,0,137882.899271,приобретение автомобиля
742,-1,-3174.456205,57,среднее,1,женат / замужем,0,F,сотрудник,0,64268.044444,дополнительное образование
800,-1,349987.852217,54,среднее,1,Не женат / не замужем,4,F,пенсионер,0,86293.724153,дополнительное образование
941,-1,-1203.369529,57,Среднее,1,женат / замужем,0,F,пенсионер,0,145017.900000,на покупку своего автомобиля
1363,-1,-1195.264956,55,СРЕДНЕЕ,1,женат / замужем,0,F,компаньон,0,69550.699692,профильное образование
1929,-1,-1461.303336,38,среднее,1,Не женат / не замужем,4,M,сотрудник,0,109121.569013,покупка жилья
2073,-1,-2539.761232,42,среднее,1,в разводе,3,F,компаньон,0,162638.609373,покупка жилья
3814,-1,-3045.290443,26,Среднее,1,гражданский брак,1,F,госслужащий,0,131892.785435,на проведение свадьбы
4201,-1,-901.101738,41,среднее,1,женат / замужем,0,F,госслужащий,0,226375.766751,операции со своей недвижимостью


Во всех 10 строках клиенты имеют среднее образование. Посмотрим внимательнее на количество детей у клиентов с таким образованием.

In [18]:
data.loc[data['education_id'] == 1].groupby('children')['children'].count()

children
-1        37
 0     10206
 1      3257
 2      1402
 3       231
 4        32
 5         8
 20       60
Name: children, dtype: int64

Количество самое разное. Отвергаем гипотезу о взаимосвязи этих параметров.

Клиенты с 20 детьми.

In [19]:
data.loc[data['children'] == 20].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,-880.221113,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья
720,20,-855.595512,44,среднее,1,женат / замужем,0,F,компаньон,0,112998.738649,покупка недвижимости
1074,20,-3310.411598,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518.537004,получение образования
2510,20,-2714.161249,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474.835577,операции с коммерческой недвижимостью
2941,20,-2161.591519,42,среднее,1,женат / замужем,0,F,сотрудник,0,199739.941398,на покупку автомобиля
3302,20,-1203.369529,35,среднее,1,Не женат / не замужем,4,F,госслужащий,0,145017.900000,профильное образование
3396,20,-1203.369529,56,высшее,0,женат / замужем,0,F,компаньон,0,145017.900000,высшее образование
3671,20,-913.161503,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,101255.492076,на покупку подержанного автомобиля
3697,20,-2907.910616,40,среднее,1,гражданский брак,1,M,сотрудник,0,115380.694664,на покупку подержанного автомобиля
3735,20,-805.044438,26,высшее,0,Не женат / не замужем,4,M,сотрудник,0,137200.646181,ремонт жилью


У все отрицательное значение в столбце days_employed. Проверим, что это не связанные события.

In [20]:
data.loc[data['days_employed'] < 0]['children'].mean()

0.617146017699115

Среднее количество детей не равно ровно 20, поэтому отвергаем гипотезу о взаимосвязи этих аномалий.

Получаем, что аномальные значения абсолютно случайны, и у нас нет возможности их восстановить. Удалим эти строки используя метод drop().

In [21]:
data = data.drop(data[data['children'] == -1].index)

In [22]:
data = data.drop(data[data['children'] == 20].index)

Проверим.

In [23]:
data['children'].value_counts()

0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

In [24]:
data['children'].count()

21402

Строки с аномальным количество детей удалены.

Убедимся, что нет скрытых пропусков в столбцах education_id, family_status_id, debt.

In [25]:
data['education_id'].value_counts()

1    15136
0     5237
2      741
3      282
4        6
Name: education_id, dtype: int64

In [26]:
data['family_status_id'].value_counts()

0    12302
1     4160
4     2799
3     1189
2      952
Name: family_status_id, dtype: int64

In [27]:
data['debt'].value_counts()

0    19670
1     1732
Name: debt, dtype: int64

Данные в этих столбцах принимают заявленные значеия.

Проверим теперь столбцы с текстовыми переменными. Начнем с образования.

In [28]:
data['education'].value_counts()

среднее                13667
высшее                  4698
СРЕДНЕЕ                  766
Среднее                  703
неоконченное высшее      665
ВЫСШЕЕ                   271
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

В столбце с данными об образовании пропущенных значений нет, зато есть дубликаты. Их мы обработаем после пропусков.

Посмотрим на семейный статус.

In [29]:
data['family_status'].value_counts()

женат / замужем          12302
гражданский брак          4160
Не женат / не замужем     2799
в разводе                 1189
вдовец / вдова             952
Name: family_status, dtype: int64

В данных о семейном статусе нет проблем, которые мешали бы анализу. Тем не менее, различие в регистре можем устранить на этапе обработки дубликатов.

Изучим данные о поле клиентов.

In [30]:
data['gender'].value_counts()

F      14154
M       7247
XNA        1
Name: gender, dtype: int64

Здесь видим одно пропущенное значение. Выведем строку с ним.

In [31]:
data.loc[data['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,-2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


Это полностью слуайный пропуск - он не зависит от других факторов в данных. Этот пропуск - единственный такого рода и не имеет явных закономерностей. Сделаем вывод, что он возник в виду человеческого фактора. У нас нет имени клиента, чтобы восстановить его пол. Пропущенное значение уже имеет тектовый формат, при этом пол клиента не будет использован для анализа. Оставим данный пропуск как есть.

Рассмотрим данные о типе занятости клиентов.

In [32]:
data['income_type'].value_counts()

сотрудник          11050
компаньон           5054
пенсионер           3839
госслужащий         1453
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

В данном столбце видим категории, которые можно объединить в одну. Например, "компаньон" и "предприниматель". Объединим категории на последнем этапе предобработки.

Изучим столбец с целями получения кредита.

In [33]:
data['purpose'].value_counts()

свадьба                                   796
на проведение свадьбы                     772
сыграть свадьбу                           769
операции с недвижимостью                  673
покупка коммерческой недвижимости         661
покупка жилья для сдачи                   651
операции с жильем                         648
операции с коммерческой недвижимостью     646
жилье                                     642
покупка жилья                             641
покупка жилья для семьи                   640
недвижимость                              632
строительство собственной недвижимости    628
операции со своей недвижимостью           626
строительство жилой недвижимости          622
строительство недвижимости                620
покупка недвижимости                      619
покупка своего жилья                      619
ремонт жилью                              609
покупка жилой недвижимости                603
на покупку своего автомобиля              504
заняться высшим образованием      

Как уже было замечено ранее, одинаковые цели внесены разными фразами. Здесь следует воспользоваться лемматизацией, после чего можно будет отнести каждую цель к более лаконичной категории.

### Вывод

В изучаемых данных были обнаружены и затем устранены пропущенные значения. Количественные переменные заполнены характерными значениями, категориальные переменные с пропусками удалены либо оставлены как есть. В ходе поиска пропусков были обнаружены другие проблемы с данными, которыми займемся далее.

### Замена типа данных

Вещественный тип данных встречается в двух столбцах - days_employed и total_income. 

В столбце total_income значения внесены с точностью до 6 знаков после запятой. Для анализа нам такая точность не нужна, а визуальному восприятию таблицы это мешает. Преобразуем данные столбца в целочисленный формат. Так как данные имеют тип float64, достаточно будет воспользоваться методом astype().

In [34]:
data['total_income'] = data['total_income'].astype('int')

В столбце days_employed есть аномально большие значения. Для нужд анализа они нам пока не понадобятся, поэтому оставим как есть. Некоторые значения внесены как отрицательные. Так же как и в предыдущем случае, значения записаны с точностью до 6 знаков после запятой. Сделаем таблицу более читаемой. Преобразуем столбец days_employed в целочисленный формат и возьмем модуль значений методом abs().

In [35]:
data['days_employed'] = data['days_employed'].astype('int').abs()

Проверим результат.

In [36]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21402 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21402 non-null  int64 
 1   days_employed     21402 non-null  int64 
 2   dob_years         21402 non-null  int64 
 3   education         21402 non-null  object
 4   education_id      21402 non-null  int64 
 5   family_status     21402 non-null  object
 6   family_status_id  21402 non-null  int64 
 7   gender            21402 non-null  object
 8   income_type       21402 non-null  object
 9   debt              21402 non-null  int64 
 10  total_income      21402 non-null  int64 
 11  purpose           21402 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


### Вывод

Произведена замена вещественного типа данных на целочисленный. Обработаны отрицательные значения. Аномально большие значения оставлены как есть, так как не будут использоваться в рамках этого анализа.

### Обработка дубликатов

Выявим возможные дубликаты в данных. Количественные данные имеют тип int64, то есть их можно сравнивать между собой. Категориальные данные могут иметь дубликаты, различающиеся регистром. Такие мы уже обнаружили в столбце education. Приведем данные этого столбца к нижнему регистру методом str.lower().

In [38]:
data['education'] = data['education'].str.lower()

Также приведем к нижнему регистру данные о семейном статусе и поле - для единообразия записей. В других столбцах записей с символами в верхнем регистре обнаружено не было.

In [39]:
data['family_status'] = data['family_status'].str.lower()

In [40]:
data['gender'] = data['gender'].str.lower()

Посчитаем полные дубликаты строк.

In [41]:
data.duplicated().sum()

72

Строка, которая полностью дублирует другую, не дает нам никакой информации. Сбросим такие строки и перестроим заново индекс.

In [42]:
data = data.drop_duplicates().reset_index(drop=True)

Проверим результат.

In [43]:
data.duplicated().sum()

0

In [44]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21330 entries, 0 to 21329
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21330 non-null  int64 
 1   days_employed     21330 non-null  int64 
 2   dob_years         21330 non-null  int64 
 3   education         21330 non-null  object
 4   education_id      21330 non-null  int64 
 5   family_status     21330 non-null  object
 6   family_status_id  21330 non-null  int64 
 7   gender            21330 non-null  object
 8   income_type       21330 non-null  object
 9   debt              21330 non-null  int64 
 10  total_income      21330 non-null  int64 
 11  purpose           21330 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


Дубликатов не осталось, количество строк в данных уменьшилось на 72.

### Вывод

Были удалены дублирующие строки в данных с учетом регистра.

Можно предположить, что дубликаты появились вследствие человеческого фактора. Запись одних и тех же данных в разном регистре - это скорее всего ручной ввод оператором. Полные дубликаты могли возникнуть как следствие того же человеческого фактора либо как следствие технического сбоя.

### Лемматизация

Как мы заметили просматривая уникальные значения столбца purpose, цели получения кредита имеют дубли, хотя записаны разными выражениями. Выделим леммы, чтобы в дальнейшем разбить цели по категориям. Для этого добавим в таблицу столбец с лемматизированными целями.

Лемматизацию будем проводить с помощью библиотеки pymystem3.

In [45]:
from pymystem3 import Mystem
m = Mystem()

ModuleNotFoundError: No module named 'pymystem3'

Запишем функцию, которая получает на вход текст, а возвращает его леммы с помощью метода m.lemmatize().

In [46]:
def make_lemmas(text):
    lemmas_list = m.lemmatize(text)
    return lemmas_list

Применим нашу функцию make_lemmas к столбцу с целями получения кредита. Результат запишем в новый столбец с леммами.

In [47]:
data['lemmas'] = data['purpose'].apply(make_lemmas)

Проверим результат.

In [48]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]"
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]"
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]"
5,0,926,27,высшее,0,гражданский брак,1,m,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]"
6,0,2879,43,высшее,0,женат / замужем,0,f,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]"
7,0,152,50,среднее,1,женат / замужем,0,m,сотрудник,0,135823,образование,"[образование, \n]"
8,2,6929,35,высшее,0,гражданский брак,1,f,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]"
9,0,2188,41,среднее,1,женат / замужем,0,m,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]"


### Вывод

Были выделены леммы в значениях столбца с целями получения кредита. Данные готовы к категоризации.

### Категоризация данных

Помним, что в данных о типе дохода у нас есть категории, которые можно объединить. Посмотрим на список еще раз.

In [49]:
data['income_type'].value_counts()

сотрудник          11014
компаньон           5047
пенсионер           3812
госслужащий         1451
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

Предпринимателей можно отнести к типу "компаньон". Студента и клиента в декрете - к типу "безработный". Выполним замену.

In [50]:
data.loc[data['income_type'] == 'предприниматель', 'income_type'] = 'компаньон'

In [51]:
data.loc[data['income_type'] == 'студент', 'income_type'] = 'безработный'

In [52]:
data.loc[data['income_type'] == 'в декрете', 'income_type'] = 'безработный'

Проверим результат.

In [53]:
data['income_type'].value_counts()

сотрудник      11014
компаньон       5049
пенсионер       3812
госслужащий     1451
безработный        4
Name: income_type, dtype: int64

Так гораздо лучше!

Определим категории для данных в соответствие с поставленными вопросами:
1. Есть ли зависимость между наличием детей и возвратом кредита в срок?
1. Есть ли зависимость между семейным положением и возвратом кредита в срок?
1. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
1. Как разные цели кредита влияют на его возврат в срок?



Количество детей само по себе и есть категория. Семейное положение - тоже. Дополнительные действия понадобятся, чтобы разделить по категориям уровень дохода и цели кредита.

Для того, чтобы выделить категории по доходам, посмотрим на минимальное, максимальное, среднее и медианное значения.

In [54]:
data['total_income'].min()

20667

In [55]:
data['total_income'].max()

2265604

In [56]:
data['total_income'].mean()

165250.380403188

In [57]:
data['total_income'].median()

145017.0

Разброс значений - от 20 тысяч до 2 миллионов. Проверим, а не выброс ли максимальное значение. Построим гистограмму.

In [58]:
print(data['total_income'].hist())

AxesSubplot(0.125,0.11;0.775x0.77)


Судя по длинному "хвосту", мы действительно имеем дело с выбросом. Учтем это при делении диапазона по категориям - воспользуемся квартилями, разбив диапазон на 4 части. Сразу запишем рассчитанные значения квартилей в переменные.

In [59]:
first_quartile = data['total_income'].quantile(.25)

In [60]:
first_quartile

107503.0

In [61]:
second_quartile = data['total_income'].quantile(.5)

In [62]:
second_quartile

145017.0

Второй квартиль, как и должно, совпадает с медианой.

In [63]:
third_quartile = data['total_income'].quantile(.75)

In [64]:
third_quartile

195836.75

Напишем функцию, которая будет получать на вход значение дохода, а на выходе давать категорию - номер четверти в соответствие с рассчитанными квартилями.

In [65]:
def quantify(income):
    if income <= first_quartile:
        return 'first_quater'
    elif first_quartile < income <= second_quartile:
        return 'second_quater'
    elif second_quartile < income <= third_quartile:
        return 'third_quarter'
    else:
        return 'fourth_quarter'

Применим функцию с столбцу с ежемесячным доходом. Результат запишем в новый столбец.

In [66]:
data['total_income_category'] = data['total_income'].apply(quantify)

Проверим результат.

In [67]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas,total_income_category
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",fourth_quarter
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",second_quater
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",third_quarter
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",fourth_quarter
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",third_quarter
5,0,926,27,высшее,0,гражданский брак,1,m,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]",fourth_quarter
6,0,2879,43,высшее,0,женат / замужем,0,f,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]",fourth_quarter
7,0,152,50,среднее,1,женат / замужем,0,m,сотрудник,0,135823,образование,"[образование, \n]",second_quater
8,2,6929,35,высшее,0,гражданский брак,1,f,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",first_quater
9,0,2188,41,среднее,1,женат / замужем,0,m,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",second_quater


In [68]:
data['total_income_category'].value_counts()

second_quater     6377
fourth_quarter    5333
first_quater      5333
third_quarter     4287
Name: total_income_category, dtype: int64

Категории для ежемесячного дохода определили.

Теперь посмотрим еще раз на список целей для получения кредита.

In [69]:
data['purpose'].value_counts()

свадьба                                   790
на проведение свадьбы                     762
сыграть свадьбу                           760
операции с недвижимостью                  672
покупка коммерческой недвижимости         658
покупка жилья для сдачи                   649
операции с жильем                         647
операции с коммерческой недвижимостью     645
жилье                                     641
покупка жилья                             640
покупка жилья для семьи                   637
недвижимость                              631
строительство собственной недвижимости    628
операции со своей недвижимостью           623
строительство жилой недвижимости          620
покупка своего жилья                      619
строительство недвижимости                619
покупка недвижимости                      616
ремонт жилью                              604
покупка жилой недвижимости                602
на покупку своего автомобиля              504
заняться высшим образованием      

Выделим следующие категории:
* свадьба
* автомобиль
* образование
* покупка недвижимости
* строительство жилья
* ремонт

Свадьбу, автомобиль и образование можно однозначно определить по совпадению с одной из лемм. Покупку, строительство и ремонт жилья можно определить хотя бы по двум словам. Ремонт + жилье = ремонт. Все остальное со словом жилье = покупка. Строительство + недвижимость = строительство жилья. Все остальное со словом недвижимость = покупка.

Напишем функцию, которая будет принимать на вход строку и искать в этой строке нужное совпадение. Возвращать функция будет категорию.


In [70]:
def purposing(row):
    lemmas = row['lemmas']
    
    if 'свадьба' in lemmas:
        return 'свадьба'
    
    elif 'автомобиль' in lemmas:
        return 'автомобиль'
    
    elif 'образование' in lemmas:
        return 'образование'
    
    elif 'недвижимость' in lemmas:
        if 'строительство' in lemmas:
            return 'строительство недвижимости'
        else:
            return 'покупка недвижимости'
        
    elif 'жилье' in lemmas:
        if 'ремонт' in lemmas:
            return 'ремонт жилья'
        else:
            return 'покупка недвижимости'
    
    else:
        return 'мде'            

Применим нашу функцию purposing к данным. Категорию цели запишем в новый столбец.

In [71]:
data['purpose_category'] = data.apply(purposing, axis=1)

Посмотрим на результат.

In [72]:
data['purpose_category'].value_counts()

покупка недвижимости          8280
автомобиль                    4279
образование                   3988
свадьба                       2312
строительство недвижимости    1867
ремонт жилья                   604
Name: purpose_category, dtype: int64

In [73]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas,total_income_category,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",fourth_quarter,покупка недвижимости
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",second_quater,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",third_quarter,покупка недвижимости
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",fourth_quarter,образование
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",third_quarter,свадьба
5,0,926,27,высшее,0,гражданский брак,1,m,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]",fourth_quarter,покупка недвижимости
6,0,2879,43,высшее,0,женат / замужем,0,f,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]",fourth_quarter,покупка недвижимости
7,0,152,50,среднее,1,женат / замужем,0,m,сотрудник,0,135823,образование,"[образование, \n]",second_quater,образование
8,2,6929,35,высшее,0,гражданский брак,1,f,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",first_quater,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,m,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",second_quater,покупка недвижимости


Покупка недвижимости - самая популярная цель кредитования, ремонт - наименее популярная. 

### Вывод

Выделили категории для целей кредитования и доходов. Можем приступать к анализу!

### Шаг 3. Отвечаем на вопросы исследования

#### Есть ли зависимость между наличием детей и возвратом кредита в срок?

Информация о возврате кредита, которой мы располагаем, следующая.

Нет долгов - 0, есть долги - 1. В качестве метрики будем считать среднее по категории. Чем ближе метрика к нулю, тем больше вероятность, что кредит будет возвращен в срок.

In [74]:
data_children = data.pivot_table(
    index='children', values='debt', aggfunc='mean')

In [75]:
data_children

,debt
children,
0,0.075444
1,0.092346
2,0.094542
3,0.081818
4,0.097561
5,0.000000


#### Вывод

Семьи с пятью детьми всегда возвращают кредит вовремя - банку стоит к ним присмотреться. Или это еще одна аномалия в данных?
Семьи с одним или тремя детьми возвращают кредит вовремя с большей вероятностью, чем семьи с двумя, тремя и четырьмя. Однако разница между этими группами незначительная. 

Вывод: количество детей незначительно влияет на возврат кредита в срок.

#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [76]:
data_family = data.pivot_table(index='family_status', values='debt', aggfunc='mean')

In [77]:
data_family

,debt
family_status,
в разводе,0.070648
вдовец / вдова,0.066246
гражданский брак,0.093153
женат / замужем,0.075606
не женат / не замужем,0.097639


#### Вывод

Познавшие замужество имеют более высокую вероятность вернуть кредит в срок.

#### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [78]:
data_income = data.pivot_table(index='total_income_category', values='debt', aggfunc='mean')

In [79]:
data_income

,debt
total_income_category,
first_quater,0.080068
fourth_quarter,0.071442
second_quater,0.085150
third_quarter,0.088873


#### Вывод

Клиенты с зарплатой в четвером квартиле имеют лучшие показатели. Опять же, незначительно. Но можно порекомендовать банку присмотреть к клиентам с доходом выше 195836,75.

#### Как разные цели кредита влияют на его возврат в срок?

In [80]:
data_purpose = data.pivot_table(index='purpose_category', values='debt', aggfunc='mean')

In [81]:
data_purpose

,debt
purpose_category,
автомобиль,0.093480
образование,0.092528
покупка недвижимости,0.072705
ремонт жилья,0.057947
свадьба,0.079152
строительство недвижимости,0.076593


#### Вывод

Желающим отремонтировать жилье банк должен быть рад больше остальных. Автомобиль и образование - цели, с которыми наибольшее количество долгов.

### Шаг 4. Общий вывод

На основе предоставленной банком статистики были получены оценки, отражающие вероятность возврата кредита в срок. Чем ближе оценка к нулю, тем выше вероятность.